# Wczytanie i przygotowanie danych

In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'C:\Users\THINK\Documents\ML\CW2\phpMYEkMl.csv')
data = data.replace('?',np.nan) 
data["CabinReduced"] =  data["cabin"].astype(str).str[0]
data["CabinReduced"] = data["CabinReduced"].replace("n",np.nan)

W moim sprawozdaniu jako zmiennych o wysokiej liczebności cech użyje 'ticket' oraz 'fare'.
W przypadku 'ticket' zastosuje zamiane zmiennej kategorycznej na liczbową w celu pozbycia sie stringów i ułatwieniu algorytmom pracy.

Jako zmiennych o niskiej liczebności cech użyje 'parch' oraz 'sibsp'

Zmienna ktorej wartosc bede starał się przewidzieć bedzie 'survived'.

In [2]:
#Zamiana zmiennej kategorycznej na liczbową
ticket = {}
for index,row in enumerate(set(data["ticket"])):
    ticket[row] = index
data["ticket_map"] = data["ticket"].map(ticket)

# RandomForestClassifier

RandomForestClassifier dokonuje predykcji na podstawie drzew decyzyjnych utworzonych z podpróbek zbioru. Dla poprawy dokładności i kontroli overfittingu domyslnie wykorzystuje metode bootstrap w celu uśrednienai wyników z poszczególnych drzew.

### Mała liczebnosc

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

#Podział zbioru
col_name = ["parch","sibsp"]
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
RF = RandomForestClassifier(n_estimators = 220, random_state = 42)
#Trenowanie modelu
RF.fit(X_train[['parch','sibsp']],y_train)
#Predykcja
PredTrain = RF.predict_proba(X_train[['parch','sibsp']])
PredTest = RF.predict_proba(X_test[['parch','sibsp']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.6471157568718543 0.6612567236425665


### Duza liczebnosc

In [37]:
col_name = ['ticket_map','fare']
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
RF = RandomForestClassifier(n_estimators = 220, random_state = 42)
#Trenowanie modelu
RF.fit(X_train[['ticket_map','fare']],y_train)
#Predykcja
PredTrain = RF.predict_proba(X_train[['ticket_map','fare']])
PredTest = RF.predict_proba(X_test[['ticket_map','fare']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)
mean_absolute_error(y_train,PredTrain[:,1])
mean_absolute_error(y_test,PredTest[:,1])

0.751673026934351 0.9904221114124555


0.34779004620434184

W przypadku danych o małej liczebności cech wyniki roc_auc_score są niezadawalające. Ponadto są one zbliżone dla zbioru testowego i treningowego co może świadczyć o niedopasowaniu.

W przypadku danych o dużej liczebności cech wyniki roc_auc_score są bardziej zadawalające, jednakże występuje duża różnica pomiędzy zbiorem testowym i treningowym co może świadczyć o nadmiernym dopasowaniu.

# LogisticRegression

Regresja logistyczna to metoda w statystyce, która jest używana do przewidywania prawdopodobieństwa wyniku i jest popularna w przypadku zadań klasyfikacji. Algorytm przewiduje prawdopodobieństwo wystąpienia zdarzenia przez dopasowanie danych do funkcji logistycznej.

### Mała liczebnosc

In [38]:
from sklearn.linear_model import LogisticRegression

#Podział zbioru
col_name = ["parch","sibsp"]
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
LR = LogisticRegression(random_state = 42)
#Trenowanie modelu
LR.fit(X_train[['parch','sibsp']],y_train)
#Predykcja
PredTrain = LR.predict_proba(X_train[['parch','sibsp']])
PredTest = LR.predict_proba(X_test[['parch','sibsp']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.5724793982633704 0.5673043539516652


### Duza liczebnosc

In [11]:
col_name = ['ticket_map','fare']
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
LR = LogisticRegression(random_state = 42)
#Trenowanie modelu
LR.fit(X_train[['ticket_map','fare']],y_train)
#Predykcja
PredTrain = LR.predict_proba(X_train[['ticket_map','fare']])
PredTest = LR.predict_proba(X_test[['ticket_map','fare']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.7073862065151264 0.6574628029727432


W przypadku danych o małej liczebności cech wyniki roc_auc_score są niezadawalające. 
Ponadto są one zbliżone dla zbioru testowego i treningowego co może świadczyć o niedopasowaniu.

W przypadku danych o dużej liczebności cech wyniki roc_auc_score również nie są zadowalające, jednakże 
różnica pomiędzy zbiorem testowym i treningowym może świadczyć o odpowiednim dopasowaniu.

# GradientBoostingClassifier

Metoda GradientBoostingClassifier buduje model predykcyjny w sposób addytywny(postępowy). Korzysta z drzew regresyjnych i pozwala na optymalizacje dowolnych różniczkowalnych funkcji strat.

### Mała liczebnosc

In [17]:
from sklearn.ensemble import GradientBoostingClassifier


#Podział zbioru
col_name = ["parch","sibsp"]
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
GBC = GradientBoostingClassifier(random_state = 42)
#Trenowanie modelu
GBC.fit(X_train[['parch','sibsp']],y_train)
#Predykcja
PredTrain = GBC.predict_proba(X_train[['parch','sibsp']])
PredTest = GBC.predict_proba(X_test[['parch','sibsp']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.6560616116365244 0.6603560448628604


### Duza liczebnosc

In [16]:
#Podział zbioru
col_name = ['ticket_map','fare']
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
GBC = GradientBoostingClassifier(random_state = 42)
#Trenowanie modelu
GBC.fit(X_train[['ticket_map','fare']],y_train)
#Predykcja
PredTrain = GBC.predict_proba(X_train[['ticket_map','fare']])
PredTest = GBC.predict_proba(X_test[['ticket_map','fare']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.7498617333112106 0.8658416314865225


W przypadku danych o małej liczebności cech wyniki roc_auc_score są niezadawalające. 
Ponadto są one zbliżone dla zbioru testowego i treningowego co może świadczyć o niedopasowaniu.

W przypadku danych o dużej liczebności cech wyniki roc_auc_score są jak dotąd najlepsze. Ponadto 
różnica pomiędzy zbiorem testowym i treningowym może świadczyć o odpowiednim dopasowaniu.

# AdaBoostClassifier

AdaBoostClassifier jest metaestymatorem, który zaczyna się od dopasowania klasyfikatora do oryginalnego zbioru danych,
a następnie dopasowuje dodatkowe kopie klasyfikatora do tego samego zbioru danych, ale w którym wagi 
niepoprawnie sklasyfikowanych instancji są dostosowywane tak, aby kolejne klasyfikatory skupiały się bardziej na 
trudnych przypadkach.

### Mała liczebnosc

In [20]:
from sklearn.ensemble import AdaBoostClassifier

#Podział zbioru
col_name = ["parch","sibsp"]
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
ABC = AdaBoostClassifier(random_state = 42)
#Trenowanie modelu
ABC.fit(X_train[['parch','sibsp']],y_train)
#Predykcja
PredTrain = ABC.predict_proba(X_train[['parch','sibsp']])
PredTest = ABC.predict_proba(X_test[['parch','sibsp']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.6523007577014546 0.6499101837082807


### Duza liczebnosc

In [21]:
#Podział zbioru
col_name = ['ticket_map','fare']
X_train, X_test, y_train, y_test = train_test_split(data[col_name], data["survived"], test_size=0.3, random_state=0)

#Tworzenie modelu
ABC = AdaBoostClassifier(random_state = 42)
#Trenowanie modelu
ABC.fit(X_train[['ticket_map','fare']],y_train)
#Predykcja
PredTrain = ABC.predict_proba(X_train[['ticket_map','fare']])
PredTest = ABC.predict_proba(X_test[['ticket_map','fare']].fillna(0))


RAS_test = roc_auc_score(y_test,PredTest[:,1])
RAS_train = roc_auc_score(y_train,PredTrain[:,1])

print(RAS_test,RAS_train)

0.7074553398595209 0.7702086656368402


W przypadku danych o małej liczebności cech wyniki roc_auc_score są niezadawalające. Ponadto są one zbliżone dla zbioru testowego i treningowego co może świadczyć o niedopasowaniu.

W przypadku danych o dużej liczebności cech wyniki roc_auc_score są nieco gorsze od poprzedniej metody jednak nadal lepsze
niż pozostałe. Ponadto różnica pomiędzy zbiorem testowym i treningowym może świadczyć o odpowiednim dopasowaniu.

# Wnioski z porównania metod

Porównując zastosowane algorytmy, dla danych o małej liczności cech, w każdym przypadku występował problem niedopasowania oraz słabych wyników modeli predykcyjnych.

Dla danych o dużej ilości cech sytuacja znacznie sie poprawiła a modele predykcyjne osiągały lepsze rezultaty. 
Algorytm RandomForestClassifier miał tendencje do overfittingu. Pozostałe algorytmy wykazywały odpowiednie dopasowanie a najlepsze rezultaty osiągnęły GradientBoostingClassifier oraz AdaBoostClassifier.

# Różnica między predict_proba a predict


predict -  zwraca binarną wartosc 0 lub 1, czyli na sztywno przewiduje nasze wartosci

predict_proba - zwraca prawdopodobienstwo binarnej wartosci 0 lub 1, czyli zamiast sztywnej wartosci zwraca prawdopodobinestwo jej wystąpienia